In [19]:
import re
import os
from fuzzywuzzy import fuzz
from nltk.tokenize import sent_tokenize, word_tokenize


def read_file(file_path):
    with open(file_path, 'r') as f:
        return f.read()

def write_file(file_path, content):
    with open(file_path, 'w') as f:
        f.write(content)


def check_match(segment, clip, min_words=8, fuzz_ratio=90):
    for sentence in clip:
        if len(word_tokenize(sentence)) >= min_words:
            for seg_sentence in segment:
                if fuzz.ratio(sentence, seg_sentence) >= fuzz_ratio:
                    return True, [sentence]
    return False, []

def find_tiktok_segments(youtube_transcript, tiktok_transcripts, threshold=90):
    tiktok_positions = []
    youtube_sentences = sent_tokenize(youtube_transcript)
    
    for tiktok in tiktok_transcripts:
        tiktok_sentences = sent_tokenize(tiktok)
        max_similarity = 0
        best_match = (0, 0)
        
        for i in range(len(youtube_sentences) - len(tiktok_sentences) + 1):
            segment = youtube_sentences[i:i + len(tiktok_sentences)]
            is_match, _ = check_match(segment, tiktok_sentences, fuzz_ratio=threshold)
            
            if is_match:
                start_pos = sum(len(s) for s in youtube_sentences[:i]) + i  # +i for spaces between sentences
                end_pos = start_pos + len(tiktok)
                tiktok_positions.append((start_pos, end_pos))
                print(f"Found TikTok segment at positions {start_pos, end_pos}")
                break
        else:
            print(f"Did not find TikTok segment: {tiktok[:30]}... in YouTube transcript.")
            
    return tiktok_positions


def custom_segment_transcript(youtube_transcript, tiktok_positions):
    segments = []
    last_end = 0
    for start, end in tiktok_positions:
        if start > last_end:
            segments.append(youtube_transcript[last_end:start])
        segments.append(youtube_transcript[start:end])
        last_end = end
    if last_end < len(youtube_transcript):
        segments.append(youtube_transcript[last_end:])
    return segments

def label_segments(segments, tiktok_positions):
    labels = []
    current_pos = 0
    labeled_segments = []
    for segment in segments:
        segment_start = current_pos
        segment_end = current_pos + len(segment)
        label = "Not TikTok-worthy"
        for start, end in tiktok_positions:
            if segment_start == start and segment_end == end:
                label = "TikTok-worthy"
                break
        labeled_segments.append(f"{label}:\n{segment}\n")
        current_pos += len(segment)
    return "".join(labeled_segments)

# File paths
youtube_transcript_path = "/Users/rossjackson/Documents/VideoAnalysisProject/New_Project_Directory/TikTok:YT_Audios/#37/#37 The Mission _ Suicide letter, Angry Jamaican man and a Dagger up the A_se-CD6TlG1s4yk_55716-10-18_Powered by notta.ai.txt"
tiktok_paths = [
    "/Users/rossjackson/Documents/VideoAnalysisProject/New_Project_Directory/TikTok:YT_Audios/#37/yt_37_1_55725-06-13_Powered by notta.ai.txt",
    "/Users/rossjackson/Documents/VideoAnalysisProject/New_Project_Directory/TikTok:YT_Audios/#37/yt_37_2_55725-06-13_Powered by notta.ai.txt",
    "/Users/rossjackson/Documents/VideoAnalysisProject/New_Project_Directory/TikTok:YT_Audios/#37/yt_37_3_55725-06-13_Powered by notta.ai.txt",
    "/Users/rossjackson/Documents/VideoAnalysisProject/New_Project_Directory/TikTok:YT_Audios/#37/yt_37_4_55725-06-13_Powered by notta.ai.txt"
]

# Read files
youtube_transcript = read_file(youtube_transcript_path)
tiktok_transcripts = [read_file(path) for path in tiktok_paths]

# Find, segment, and label
tiktok_positions = find_tiktok_segments(youtube_transcript, tiktok_transcripts)
segments = custom_segment_transcript(youtube_transcript, tiktok_positions)
labeled_segments = label_segments(segments, tiktok_positions)

# Write to file
output_path = os.path.splitext(youtube_transcript_path)[0] + "_labeled.txt"
labeled_content = label_segments(segments, tiktok_positions)
write_file(output_path, labeled_content)

print(f"Labeled segments have been written to {output_path}")


Found TikTok segment at positions (5213, 7055)
Found TikTok segment at positions (42474, 43393)
Found TikTok segment at positions (72289, 73270)
Did not find TikTok segment: 
Holly will it be and Phillips... in YouTube transcript.
Labeled segments have been written to /Users/rossjackson/Documents/VideoAnalysisProject/New_Project_Directory/TikTok:YT_Audios/#37/#37 The Mission _ Suicide letter, Angry Jamaican man and a Dagger up the A_se-CD6TlG1s4yk_55716-10-18_Powered by notta.ai_labeled.txt
